In [1]:
from sentinelhub import SHConfig


INSTANCE_ID = ''  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

In [2]:
import datetime
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sentinelhub import WcsRequest, MimeType, CRS, BBox

In [17]:
#initialize wcs request

def get_bands_request(coords, year):
    
    #set date range
    date1 = year + '-03-01'
    date2 = year + '-04-30'
    
    #make bounding box
    box = BBox(bbox=coords, crs=CRS.WGS84)
    
    #specify request arguments
    wcs_bands_request = WcsRequest(
        layer='BANDS-S2-L1C',
        bbox=box,
        time=(date1, date2),
        resx = '10m',
        resy = '10m',
        image_format=MimeType.TIFF_d32f,
        time_difference=datetime.timedelta(hours=2),
        maxcc = 0.9,
        config=config
    )
    
    return wcs_bands_request

In [5]:
#get images
def get_images(wcs_bands_request):
    
    wcs_bands_img = wcs_bands_request.get_data()
    
    return wcs_bands_img

In [6]:
#save images

def save_images(wcs_bands_request, wcs_bands_img, location, year):
    
    #get dates of images and convert to strings
    dates = wcs_bands_request.get_dates()
    date_strings = [datetime.datetime.strftime(date, '%Y-%m-%d') for date in dates]
    
    #save each 3D array as a separate file
    for name, img in zip(date_strings, wcs_bands_img):
        
        print(name)
        
        #build filename
        filename = '/home/esther/shared/' + name + '_' + location
        
        #save as .npy file
        np.save(filename, img)

In [7]:
#set bounding box coordinates for different locations

#Antelope Valley
antelope_coords = [-118.4, 34.72, -118.32, 34.79]

#Lake Elsinore
elsinore_coords = [-117.34, 33.72, -117.44, 33.8]

#Grass Mountain
grassmtn_coords = [-120.02, 34.75, -120.11, 34.82]

#Carrizo Plains - for another time
carrizo_coords = [-119.59, 35.05, -119.69, 35.14]

In [23]:
#put it all together

#change these variables as needed
location = 'antelope'
coords = antelope_coords
year = '2017'

#set up request
%time wcs_bands_request = get_bands_request(coords, year)

CPU times: user 37.3 ms, sys: 0 ns, total: 37.3 ms
Wall time: 884 ms


In [24]:
#get images
%time wcs_bands_img = get_images(wcs_bands_request)

CPU times: user 1.81 s, sys: 810 ms, total: 2.62 s
Wall time: 31.8 s


In [25]:
#save images
save_images(wcs_bands_request, wcs_bands_img, location, year)

2017-03-05
2017-03-15
2017-03-25
2017-04-04
2017-04-14
2017-04-24
